In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/americannlp-task-2-dataset/maya-test.tsv
/kaggle/input/americannlp-task-2-dataset/bribri-dev.tsv
/kaggle/input/americannlp-task-2-dataset/nahuatl_omitlan-test.tsv
/kaggle/input/americannlp-task-2-dataset/nahuatl_omitlan-dev.tsv
/kaggle/input/americannlp-task-2-dataset/guarani-train.tsv
/kaggle/input/americannlp-task-2-dataset/guarani-dev.tsv
/kaggle/input/americannlp-task-2-dataset/guarani-test.tsv
/kaggle/input/americannlp-task-2-dataset/maya-train.tsv
/kaggle/input/americannlp-task-2-dataset/bribri-test.tsv
/kaggle/input/americannlp-task-2-dataset/maya-dev.tsv
/kaggle/input/americannlp-task-2-dataset/nahuatl_omitlan-train.tsv
/kaggle/input/americannlp-task-2-dataset/bribri-train.tsv


In [2]:
%%capture
%pip install Dataset
%pip install sacrebleu
%pip install transformers
%pip install sentencepiece
%pip install datasets
%pip install huggingface_hub
%pip install bitsandbytes
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [3]:
# Set environment variable to help with memory allocation
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HUGGINGFACE_TOKEN")

!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `basic task` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `basic task`


In [5]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer,SFTConfig
from trl import setup_chat_format
from transformers import (
                          AutoTokenizer,
                          AutoModelForCausalLM,
                          TrainingArguments,
                          BitsAndBytesConfig,
                          pipeline,
                          Trainer,
                          DataCollatorWithPadding,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split
from sacrebleu import corpus_bleu, corpus_chrf

In [6]:
from accelerate import PartialState
device_map={"": PartialState().process_index}

In [7]:
# Load the data
train_df = pd.read_table('/kaggle/input/americannlp-task-2-dataset/guarani-train.tsv')
dev_df = pd.read_table('/kaggle/input/americannlp-task-2-dataset/guarani-dev.tsv')
test_df = pd.read_table('/kaggle/input/americannlp-task-2-dataset/guarani-test.tsv')

In [8]:
X_train = train_df
X_eval = dev_df
X_test_sub = test_df

In [9]:
X_train

,ID,Source,Change,Target
0,Guarani0057,Ha’e ombojerekuri umi kutuhakuéra poro’o,PERSON:2_SI,Nde rembojerekuri umi kutuhakuéra tuicháva
1,Guarani0058,Ha’e ombojerekuri umi kutuhakuéra poro’o,TYPE:NEG,Ha’e ndombojereikuri umi kutuhakuéra tuicháva
2,Guarani0059,Ha’e ombojerekuri umi kutuhakuéra poro’o,ASPECT:IPFV,Ha’e ombojerehina kuri umi kutuhakuéra tuicháva
3,Guarani0197,Mombe’ukuéra omboty kuri pende arete,TYPE:NEG,Mombe’ukuéra ndombotyi kuri pende arete
4,Guarani0198,Mombe’ukuéra omboty kuri pende arete,ASPECT:IPFV,Mombe’ukuéra omboty kuri hína pende arete
...,...,...,...,...
173,Guarani0043,Che rasy,PERSON:2_SI,Nde nderasy
174,Guarani0044,Che rasy,PERSON:3_SI,Ha’e hasy
175,Guarani0045,Che rasy,PERSON:1_PL_INC,Ñande rasy
176,Guarani0046,Che rasy,PERSON:1_PL_EXC,Ore rorasy


## Transfer Tags to Instruction

In [59]:
def translate_tags_to_instruction(change):
    """
    Translate the tags in the 'Change' field to full-form instructions, combining multiple instructions with 'and.'
    """
    # Split the "Change" field into tags
    instruction_tags = change.split(", ")
    instructions = []

    # Define mapping dictionaries for each category
    type_map = {
        "NEG": "Make the sentence negative",
        "IMP": "Change the sentence to imperative mood",
        "AFF": "Make the sentence affirmative"
    }

    mode_map = {
        "DES": "Express desire or wish to perform the action",
        "EXH": "Change to exhortative mood (encouraging or urging action)",
        "ADVERS": "Express that the action was done despite difficulties",
        "POT": "Express potential or ability to perform the action"
    }

    tense_map = {
        "IPFV_HAB": "Change to habitual imperfective aspect",
        "IPFV_REC": "Change to recent imperfective aspect",
        "IPFV_PROG": "Change to progressive imperfective aspect",
        "PRF_PROG": "Change to perfect progressive aspect",
        "PRF_REC": "Change to recent perfect tense",
        "FUT_POT": "Change to potential future tense",
        "FUT_CER": "Change to certain future tense",
        "PAS_PLU": "Change to pluperfect (past perfect) tense"
    }

    aspect_map = {
        "IPFV": "Change to imperfective aspect",
        "PFV": "Change to perfective aspect",
        "INC": "Express the beginning or initiation of the action"
    }

    voice_map = {
        "MID": "Change to middle voice"
    }

    absnum_map = {
        "PL": "Make the absolutive argument plural",
        "NI": "Remove number marking from the absolutive argument"
    }

    person_map = {
        "[SUBJ]:1_PL_EXC": "Change subject to first person plural exclusive",
        "[SUBJ]:1_PL_INC": "Change subject to first person plural inclusive",
        "[SUBJ]:2_PL": "Change subject to second person plural",
        "[SUBJ]:3_PL": "Change subject to third person plural",
        "[SUBJ]:2_SI": "Change subject to second person singular",
        "[SUBJ]:3_SI": "Change subject to third person singular",
        "[SUBJ]:NA": "Remove person marking"
    }

    # Translate each tag into a full-form instruction
    for tag in instruction_tags:
        category, value = tag.split(":")
        
        if category == "TYPE":
            instructions.append(type_map.get(value.strip(), f"Unknown TYPE: {value.strip()}"))
        elif category == "MODE":
            instructions.append(mode_map.get(value.strip(), f"Unknown MODE: {value.strip()}"))
        elif category == "TENSE":
            instructions.append(tense_map.get(value.strip(), f"Unknown TENSE: {value.strip()}"))
        elif category == "ASPECT":
            instructions.append(aspect_map.get(value.strip(), f"Unknown ASPECT: {value.strip()}"))
        elif category == "VOICE":
            instructions.append(voice_map.get(value.strip(), f"Unknown VOICE: {value.strip()}"))
        elif category == "ABSNUM":
            instructions.append(absnum_map.get(value.strip(), f"Unknown ABSNUM: {value.strip()}"))
        elif category.startswith("PERSON"):
            instructions.append(person_map.get(category + ":" + value.strip(), f"Unknown PERSON: {value.strip()}"))
        else:
            instructions.append(f"Unknown category: {category} with value {value.strip()}")
    
    # Combine all instructions with 'and'
    return ' and '.join(instructions)

In [19]:
# # Example usage
# data_point = {
#     "Source": "Ye' shka'",
#     "Change": "TYPE:NEG, TENSE:IPFV_REC, PERSON:1_PL_EXC",
#     "Target": "Kë̀ ye' shkö̀"
# }

# instruction = translate_tags_to_instruction(data_point)
# print(instruction)

## Generate Prompt

In [39]:
# Define functions for generating prompts
def generate_prompt(row):
    """
    Generate a structured training prompt for a given data point.
    """
    instruction = translate_tags_to_instruction(row["Change"])
    return (
        f"Language: Guarani\n"
        f"Task: Transform the Source sentence into the Target sentence based on the given instruction.\n\n"
        f"Instruction: {instruction}\n"
        f"Source: {row['Source']}\n"
        f"Target: {row['Target']}"
    )

def generate_test_prompt(row):
    """
    Generate a structured test prompt for a given data point.
    """
    instruction = translate_tags_to_instruction(row["Change"])
    return (
        f"Language: Guarani\n"
        f"Task: Transform the Source sentence into the Target sentence based on the given instruction.\n\n"
        f"Instruction: {instruction}\n"
        f"Source: {row['Source']}\n"
        f"Provide only the transformed Target sentence."
    )

In [13]:
# # zero sort prompting type 1
# def generate_prompt(data_point):
#     """
#     Generate a structured training prompt for a given data point.
#     """
#     return f"""Language: Guarani, Rewrite and change the following sentence according to given instruction.
# Instruction: {data_point["Change"]}
# Source: {data_point["Source"]}
# Target: {data_point["Target"]}
# """.strip()

# def generate_test_prompt(data_point):
#     return f"""
# Language: Guarani, Rewrite and change the following sentence according to given instruction.
# Instruction: {data_point["Change"]}
# Source: {data_point["Source"]}
# Provide only the Target sentence nothing else.
# Target:""".strip()

In [40]:
# Generate prompts for training and evaluation data
X_train["text"] = X_train.apply(generate_prompt, axis=1)
X_eval["text"] = X_eval.apply(generate_prompt, axis=1)

In [41]:
# # Generate prompts for training and evaluation data
# X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
# X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

In [42]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [43]:
# Create a new DataFrame for test prompts
test_data = pd.DataFrame({
    "Change": X_eval["Change"],
    "Source": X_eval["Source"]
})
# Generate prompts for test data
X_test = pd.DataFrame(test_data.apply(lambda row: generate_test_prompt(row), axis=1), columns=["text"])

In [44]:
# Set the device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [45]:
device

device(type='cuda', index=0)

In [46]:
# Load the pre-trained model and tokenizer
base_model_name = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
# Move the model to the GPU
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e

In [ ]:
# # Define a custom predict function
# def predict(test, model, tokenizer):
#     y_pred = []
    
#     for i in tqdm(range(len(test))):
#         prompt = test.iloc[i]["text"]
#         inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
        
#         # Generate text using the model directly
#         outputs = model.generate(**inputs, max_length=100, num_beams=4, no_repeat_ngram_size=3).to(device)
        
#         generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True).to(device)
#         transformed_sentence = generated_text.split("Target:")[-1].strip()

#         if transformed_sentence:  
#             y_pred.append(transformed_sentence)
#         else:
#             y_pred.append("ERROR")  # Handle empty outputs
    
#     return y_pred

In [60]:
from tqdm import tqdm
from transformers import pipeline
import string

def clean_prediction(text):
    """
    Extracts the expected transformed sentence from the generated output.
    Stops processing as soon as a punctuation mark is encountered.
    """
    text = text.strip()

    # Extract text after "Target:" marker
    if "Target:" in text:
        text = text.split("Target:")[-1].strip()

    # Take only the first line to remove unwanted repetitions
    text = text.split("\n")[0].strip()

    # Stop at the first punctuation (e.g., period, comma, etc.)
    for punctuation in string.punctuation:
        if punctuation in text:
            text = text.split(punctuation)[0].strip()
            break

    return text

def predict(test, model, tokenizer):
    """
    Generate predictions for the test dataset without using a dataset format.
    """
    y_pred = []
    
    # Define pipeline outside loop for efficiency
    pipe = pipeline(task="text-generation", 
                    model=model, 
                    tokenizer=tokenizer, 
                    max_new_tokens=20,  # Limit length to avoid extra output
                    temperature=0.1,  # Make output more deterministic
                    )  

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]  # Use already pre-generated test prompts
        result = pipe(prompt)
        
        generated_text = result[0]['generated_text']
        transformed_sentence = clean_prediction(generated_text)
        y_pred.append(transformed_sentence if transformed_sentence else "ERROR")  # Handle empty output

    return y_pred

In [49]:
# Evaluate the model before fine-tuning
y_pred_before_fine_tune = predict(X_test, model, tokenizer)

Device set to use cuda:0
100%|██████████| 79/79 [01:00<00:00,  1.31it/s]


In [50]:
# Evaluate the predictions
def evaluate(y_true, y_pred):
    bleu = corpus_bleu(y_pred, [y_true])
    print(f"BLEU score: {bleu.score:.2f}")

    chrf = corpus_chrf(y_pred, [y_true])
    print(f"chrF score: {chrf.score:.2f}")

    for i in range(min(5, len(y_true))):
        print(f"\nMain Prompt: {X_test.iloc[i]['text']}")
        print(f"Expected Sentence: {y_true[i]}")
        print(f"Prediction: {y_pred[i]}")

In [51]:
# Evaluate the model
y_true = X_eval["Target"]

# Evaluate the model before fine-tuning
print("\nOriginal Model Evaluation Before Fine Tuning:")
evaluate(y_true.tolist(), y_pred_before_fine_tune)


Original Model Evaluation Before Fine Tuning:
BLEU score: 8.47
chrF score: 51.63

Main Prompt: Language: Guarani
Task: Transform the Source sentence into the Target sentence based on the given instruction.

Instruction: Make the sentence affirmative
Source: Ore ndorombyai kuri
Provide only the transformed Target sentence.
Expected Sentence: Ore rombyai kuri
Prediction: Ore ndorombyai kuri.

Main Prompt: Language: Guarani
Task: Transform the Source sentence into the Target sentence based on the given instruction.

Instruction: Unknown TENSE: FUT_SIM
Source: Ore ndorombyai kuri
Provide only the transformed Target sentence.
Expected Sentence: Ore ndorombyaita
Prediction: Ore ndorombyai kuri.

Main Prompt: Language: Guarani
Task: Transform the Source sentence into the Target sentence based on the given instruction.

Instruction: Change subject to first person plural inclusive
Source: Ore ndorombyai kuri
Provide only the transformed Target sentence.
Expected Sentence: Ñande nañambyai kuri


In [52]:
# Define LoRA configuration
def find_all_linear_names(model):
    cls = torch.nn.Linear
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [53]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

In [54]:
# Set up training arguments
training_arguments = SFTConfig(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=1000,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False,
    logging_steps=500,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
    eval_strategy="steps",
    eval_steps=50,  
)

In [55]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=training_arguments,
)

<ipython-input-55-1a90b2d52780>:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/178 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/178 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/178 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/178 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/79 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/79 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/79 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/79 [00:00<?, ? examples/s]

In [56]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
50,No log,1.314274
100,No log,1.263875
150,No log,1.264559
200,No log,1.295768
250,No log,1.314627
300,No log,1.355332
350,No log,1.366701
400,No log,1.360080


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=445, training_loss=0.980946195795295, metrics={'train_runtime': 408.7522, 'train_samples_per_second': 2.177, 'train_steps_per_second': 1.089, 'total_flos': 920921477314560.0, 'train_loss': 0.980946195795295})

In [61]:
# Evaluate the model after fine-tuning
y_pred_after_fine_tune = predict(X_test, model, tokenizer)

Device set to use cuda:0
100%|██████████| 79/79 [01:38<00:00,  1.25s/it]


In [62]:
# Evaluate the model
y_true = X_eval["Target"]

# Evaluate the model before fine-tuning
print("\nOriginal Model Evaluation After Fine Tuning:")
evaluate(y_true.tolist(), y_pred_after_fine_tune)


Original Model Evaluation After Fine Tuning:
BLEU score: 22.99
chrF score: 58.50

Main Prompt: Language: Guarani
Task: Transform the Source sentence into the Target sentence based on the given instruction.

Instruction: Make the sentence affirmative
Source: Ore ndorombyai kuri
Provide only the transformed Target sentence.
Expected Sentence: Ore rombyai kuri
Prediction: Language

Main Prompt: Language: Guarani
Task: Transform the Source sentence into the Target sentence based on the given instruction.

Instruction: Unknown TENSE: FUT_SIM
Source: Ore ndorombyai kuri
Provide only the transformed Target sentence.
Expected Sentence: Ore ndorombyaita
Prediction: Ore ndorombyáita kuri

Main Prompt: Language: Guarani
Task: Transform the Source sentence into the Target sentence based on the given instruction.

Instruction: Change subject to first person plural inclusive
Source: Ore ndorombyai kuri
Provide only the transformed Target sentence.
Expected Sentence: Ñande nañambyai kuri
Prediction:

## Dev Submission

In [63]:
dev_pd = pd.DataFrame(y_pred_after_fine_tune, columns=['Values'])

In [64]:
dev_pd.to_csv('syntax_squad_guarani_dev_output.tsv', sep='\t', index=False, header=False)

In [65]:
from IPython.display import FileLink

FileLink("syntax_squad_guarani_dev_output.tsv")

/kaggle/working/syntax_squad_guarani_dev_output.tsv

## Test Submission

In [66]:
# Create a new DataFrame for test prompts
test_data_sub = pd.DataFrame({
    "Change": X_test_sub["Change"],
    "Source": X_test_sub["Source"]
})
# Generate prompts for test data
X_test_sub = pd.DataFrame(test_data_sub.apply(lambda row: generate_test_prompt(row), axis=1), columns=["text"])

In [67]:
# Evaluate the model before fine-tuning
y_pred_test = predict(X_test_sub, model, tokenizer)

Device set to use cuda:0
100%|██████████| 364/364 [07:27<00:00,  1.23s/it]


In [68]:
test_pd = pd.DataFrame(y_pred_test, columns=['Values'])

In [69]:
test_pd.to_csv('syntax_squad_guarani_test_output.tsv', sep='\t', index=False, header=False)

In [70]:
from IPython.display import FileLink

FileLink("syntax_squad_guarani_test_output.tsv")

/kaggle/working/syntax_squad_guarani_test_output.tsv